# Plot the sound speed of the SCvH equation of state

### Import modules

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import interpolate as interp
import numpy as np

In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'

In [ ]:
mpl.rcParams['figure.dpi'] = 100

It seems that the sound speed $c_{s} \left( \rho, T \right)$ is not provided for the SCvH EOS. It can be calculated from
$$ c_{s} \left( \rho, T \right) = \left. \frac{\partial P}{\partial \rho} \right|_{T} + \frac{T}{\rho^2 c_{v}} \left( \left. \frac{\partial P}{\partial T} \right|_{\rho} \right)^2 $$
but that would require accurate derivatives. For the ideal (perfect) gas with
$$ P \left( \rho, T \right) = \frac{k_B}{\mu m_{H}} \rho T $$
and
$$ u \left( \rho, T \right) = c_V T$$
with
$$ c_V = \frac{k_B}{\left( \gamma - 1 \right) \mu m_{H}} $$
this expression results in
$$ c_{s} \left( \rho, T \right) = \gamma \left( \gamma - 1 \right) c_V T $$
which is indeed the sound speed of an ideal (perfect gas).

In [ ]:
def cs_ideal_gas(T, gamma=5.0/3.0, mu= 2.3):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    
    cv = kB/((gamma-1.0)*mu*m_H)
    
    return np.sqrt(gamma*(gamma-1.0)*cv*T)

In [ ]:
data = np.loadtxt("scvh_hhe_y0.275_dt_cgs_lowrhot.csv", delimiter=",", skiprows=0) 

logT_table   = data[:, 0] 
logrho_table = data[:, 1]
logP_table   = data[:, 2]
logu_table   = data[:, 3]
logs_table   = data[:, 4]

# All SCVH EOS tables obtained from Ravit have the same size
#nRho = 201
#nT   = 100
nRho = 80
nT   = 48

logrho_table = logrho_table[0:nRho]
logT_table = logT_table[0:np.size(logT_table):nRho]

logrho_min = np.min(logrho_table)
logrho_max = np.max(logrho_table)
logT_min   = np.min(logT_table)
logT_max   = np.max(logT_table)
    
dlogrho = logrho_table[1:]-logrho_table[:-1]
dlogT = logT_table[1:]-logT_table[:-1]

print("logrho_min = {:}".format(logrho_min))
print("logrho_max = {:}".format(logrho_max))
print("logT_min   = {:}".format(logT_min))
print("logT_max   = {:}".format(logT_max))
print()
#print("logrho_table = {:}".format(logrho_table))
#print("logT_table   = {:}".format(logT_table))
#print()

# Mark where the original SCvH EOS table was extended below T=100K
index = np.min(np.where(logT_table>=2.0))
print("The EOS table was extended until index={:} logT={:}".format(index, logT_table[index]))

# Split into arrays of constant T
logP_array = np.split(logP_table, nT)
logu_array = np.split(logu_table, nT)
logs_array = np.split(logs_table, nT)

# Generate 2d arrays
logP = np.vstack(logP_array)
logu = np.vstack(logu_array)
logs = np.vstack(logs_array)

# Load sound speed for the eos table
logcs = np.loadtxt("testscvheossoundspeed_table.txt")

In [ ]:
# Load sound speed table
logrho_int = np.loadtxt("testscvheossoundspeed_rhoaxis.txt")
logT_int = np.loadtxt("testscvheossoundspeed_taxis.txt")

logcs_int = np.loadtxt("testscvheossoundspeed.txt")

rho_int = 10**logrho_int
T_int = 10**logT_int

nRho_int = np.size(logrho_int)
nT_int = np.size(logT_int)

print("nRho_int= {:} nT_int= {:}".format(nRho_int, nT_int))


In [ ]:
# Skip some curves otherwise the plots are very hard to read
nSkipT = 4
nSkipRho = 8

nSkipT = 1
nSkipRho = 1

In [ ]:
# Plot log(cs) for the eos table and interpolated values
fig, ax = plt.subplots(2, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 2*y)

# cs(rho, T=const)
ax[0][0].set_prop_cycle(None)

for i in range(0, nT, nSkipT):
    ax[0][0].plot(logrho_table, logcs[i,:], '-')

ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(cs) [cm s$^{-1}$]")
ax[0][0].set(title="SCvH EOS (H-He, T=const.)")

# cs(rho=const, T)
ax[0][1].set_prop_cycle(None)

for i in range(0, nRho, nSkipRho):
    ax[0][1].plot(logT_table, logcs[:,i], '-')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(cs) [cm s$^{-1}$]")
ax[0][1].set(title="SCvH EOS H-He (rho=const.)")

# cs(rho, T=const) for interpolated values
ax[1][0].set_prop_cycle(None)

for i in range(0, nT, nSkipT):
    ax[1][0].plot(logrho_table, logcs[i,:], '-')

for i in range(0, nT_int, nSkipT):
    ax[1][0].plot(logrho_int, logcs_int[i,:], '--')
    
ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(cs) [cm s$^{-1}$]")
ax[1][0].set(title="SCvH EOS (H-He, T=const.)")

# cs(rho=const, T) for interpolated values
ax[1][1].set_prop_cycle(None)

for i in range(0, nRho, nSkipRho):
    ax[1][1].plot(logT_table, logcs[:,i], '-')

for i in range(0, nRho_int, nSkipRho):
    ax[1][1].plot(logT_int, logcs_int[:,i], '--')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(cs) [cm s$^{-1}$]")
ax[1][1].set(title="SCvH EOS H-He (rho=const.)")


In [ ]:
# Plot log(cs) for the eos table
fig, ax = plt.subplots(1, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, y)

# cs(rho, T=const)
ax[0].set_prop_cycle(None)

for i in range(0, nT, nSkipT):
    ax[0].plot(logrho_table, logcs[i,:], '-')

for i in range(0, nT, nSkipT):
    ax[0].plot(logrho_table, np.log10(np.ones(nRho)*cs_ideal_gas(10**logT_table[i])), '--')
    
ax[0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(cs) [cm s$^{-1}$]")
ax[0].set(title="SCvH EOS H-He, T=const.)")

# cs(rho=const., T)
ax[1].set_prop_cycle(None)

for i in range(0, nRho, nSkipRho):
    ax[1].plot(logT_table, logcs[:,i], '-')

for i in range(0, nRho, nSkipRho):
    ax[1].plot(logT_table, np.log10(cs_ideal_gas(10**logT_table)), '--')

ax[1].set(xlabel="log(T) [K]", ylabel="log(cs) [cm s$^{-1}$]")
ax[1].set(title="SCvH EOS H-He (rho=const.)")
